#  Importing Libraries

In [1]:
#importing necessary libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Reading the data

In [2]:
data = pd.read_csv('Loan data cleaned.csv')

#Printing first five rows
data.head()

,loannumber,loanamount,totaldue,termdays,good_bad_flag,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,employment_status_clients,...,ploan_additional charges_50%_x,ploan_additional charges_75%_x,ploan_additional charges_max_x,ploan_additional charges_mean_y,ploan_additional charges_min_y,ploan_additional charges_25%_y,ploan_additional charges_50%_y,ploan_additional charges_75%_y,ploan_additional charges_max_y,not_repaid_percentage
0,12,30000.0,34500.0,30,Good,Other,3.432010,6.433055,Diamond Bank,Permanent,...,3800.000000,4500.000000,4500.000000,3900.000000,3000.000000,3400.000000,3800.000000,4500.000000,4500.000000,64.000000
1,2,15000.0,17250.0,30,Good,Savings,3.885298,7.320700,GT Bank,Permanent,...,2665.558293,3037.525723,3543.851778,2730.359972,2102.661161,2364.738902,2665.558293,3037.525723,3543.851778,63.573297
2,7,20000.0,22250.0,15,Good,Other,11.139350,10.292041,EcoBank,Permanent,...,1500.000000,1500.000000,3000.000000,1750.000000,1500.000000,1500.000000,1500.000000,1500.000000,3000.000000,50.000000
3,3,10000.0,11500.0,15,Good,Savings,3.985770,7.491708,First Bank,Permanent,...,2250.000000,2625.000000,3000.000000,2250.000000,1500.000000,1875.000000,2250.000000,2625.000000,3000.000000,0.000000
4,9,40000.0,44000.0,30,Good,Other,7.457913,9.076574,GT Bank,Permanent,...,3800.000000,5100.000000,9000.000000,4800.000000,3000.000000,3000.000000,3800.000000,5100.000000,9000.000000,100.000000


In [3]:
data.shape

(4368, 64)

In [4]:
#make dummies of categorical columns('bank_account_type','bank_name_clients','employment_status_clients')
data=pd.get_dummies(data=data,columns=['bank_account_type','bank_name_clients','employment_status_clients'],drop_first=True)

In [5]:
x = data.drop(['good_bad_flag'], axis=1)
y = data['good_bad_flag'].replace({'Good':0, 'Bad':1})

In [6]:
#importing train_test_split
from sklearn.model_selection import train_test_split

In [7]:
#import train-test split
from sklearn.model_selection import train_test_split as tts

#divide into train and test sets
train_x, test_x, train_y, test_y = tts (x, y, random_state =101 , stratify = y)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((3276, 80), (1092, 80), (3276,), (1092,))

In [8]:
#Scalig the train and testing datasets
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
train_x = pd.DataFrame(train_x_scaled, columns=train_x.columns)

test_x_scaled = scaler.transform(test_x)
test_x = pd.DataFrame(test_x_scaled, columns=test_x.columns)

In [9]:
#importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [10]:
from sklearn.metrics import f1_score

# Model Training and Prediction

In [11]:
def blending(model, train_x, train_y, test_x):
    #train validation split
    x_train, x_valid, y_train, y_valid = tts(train_x, train_y, random_state=9)
    
    #fit on training
    model.fit(x_train, y_train)
    
    #predict on validation and test
    valid_pred = model.predict(x_valid)
    score=f1_score(y_valid,valid_pred)
    print('Validation Score', score)
    
    test_pred=model.predict(test_x)
    #return validation pred, test pred
    return valid_pred, y_valid, test_pred

In [12]:
#Model1 = Logistic Regression
LR=LogisticRegression(class_weight='balanced',random_state=101)
M1_valid, M1_target, M1_test = blending(LR, train_x, train_y, test_x)

Validation Score 0.4431137724550898


In [13]:
#Model 2= Decision Tree
DT=DecisionTreeClassifier(class_weight='balanced',random_state=101,criterion='gini',max_depth=4,max_leaf_nodes=15,min_samples_split=30)
M2_valid, M2_target, M2_test = blending(DT, train_x, train_y, test_x)

Validation Score 0.45


In [14]:
#Model 3 = K Nearest Neighbour
knn=KNeighborsClassifier()
M3_valid, M3_target, M3_test = blending(knn, train_x, train_y, test_x)

Validation Score 0.3333333333333333


In [15]:
valid_prediction = {
              'LR': M1_valid,
              'DT': M2_valid,
              'knn': M3_valid
              }
new_train = pd.DataFrame(valid_prediction)
new_train.head()

,LR,DT,knn
0,1,1,1
1,0,0,0
2,1,1,1
3,0,0,0
4,0,0,0


In [16]:
test_prediction = {
              'LR': M1_test,
              'DT': M2_test,
              'knn': M3_test
              }
new_test = pd.DataFrame(test_prediction)
new_test.head()

,LR,DT,knn
0,1,1,0
1,0,0,0
2,0,0,0
3,0,1,0
4,0,0,1


-KNN final

In [18]:
# Final Model
model1 = KNeighborsClassifier()
model1.fit(new_train, M1_target)
pred_new_test=model1.predict(new_test)
f1_score(test_y,pred_new_test)

0.41632653061224495

-Decision Tree Final

In [19]:
#FINAL MODEL
model3=DecisionTreeClassifier(class_weight='balanced',random_state=101,criterion='gini',max_depth=4,max_leaf_nodes=15,min_samples_split=30)
model3.fit(new_train, M1_target)
pred_new_test=model3.predict(new_test)
f1_score(test_y,pred_new_test)

0.3944636678200692

--Logistic

In [21]:
#Final Model
model2 = LogisticRegression(random_state=101,class_weight='balanced')
model2.fit(new_train, M1_target)
pred_new_test=model2.predict(new_test)
f1_score(test_y,pred_new_test)

0.3959731543624161